# Supervised learning Neuron Classfication

This notebook has implemented the following supervised learning neuron classifications:

- [] Pyramidal vs. Interneurons
- [] Excitatory vs. Inhibitory neurons

Datasets have been obtained from the open-source database:
Mizuseki, K., Sirota, A., Pastalkova, E., Diba, K. & Buzsaki, G. Multiple single unit recordings from different rat hippocampal and entorhinal regions while the animals were performing multiple behavioral tasks. CRCNS.org (2013). \
URL http://dx.doi.org/10.6080/K09G5JRZ.

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.constraints import max_norm
import os
import sys

cwd = os.getcwd()

root_folder = os.sep+"ML-Neuron-Classification"
sys.path.insert(0, cwd[:(cwd.index(root_folder)+len(root_folder))] + os.sep)

from utils.fully_connected_ANN import*
from utils.unpacking_dat_files import*
from utils.helper_functions import*

In [2]:
# Load data
fileNameWF = 'C:/Users/Marius/Documents/Datasets/.dat_testing/ec014_Spikes__Waveform.bin'
fileNamePars = 'C:/Users/Marius/Documents/Datasets/.dat_testing/ec014_Spikes__Parameters.bin'

data = ReadBinWaveform(fileNameWF)
parameters = ReadBinParameters(fileNamePars)

print(data.shape)
print(parameters.shape)

(81930, 256)
(81930, 11)


## 1) Classification in Pyramidal vs. Interneurons

In [4]:
#Create labels from parameters file
y = creating_y_labels_from_parameters_file(parameters, 'PyramidalvsInter')

#Shuffle Waveforms and respective labels
data = np.append(data, y, axis=1)
reduced_data = data[data[:,256] != 2]  #Delete 'neither'
reduced_data = np.take(reduced_data,np.random.permutation(reduced_data.shape[0]),axis=0,out=reduced_data) #random shuffle

#Divide data in training, dev & test set (61055 + 10000 + 10000)
X_train = reduced_data[0:(reduced_data.shape[0]-20000),0:256]
Y_train = reduced_data[0:(reduced_data.shape[0]-20000),256]

X_dev = reduced_data[(reduced_data.shape[0]-20000):(reduced_data.shape[0]-10000),0:256]
Y_dev = reduced_data[(reduced_data.shape[0]-20000):(reduced_data.shape[0]-10000),256]

X_test = reduced_data[(reduced_data.shape[0]-10000):(reduced_data.shape[0]),0:256]
Y_test = reduced_data[(reduced_data.shape[0]-10000):(reduced_data.shape[0]),256]

print("Shape X_train: ", X_train.shape)
print("Shape Y_train: ", Y_train.shape)

print("Shape X_dev: ", X_dev.shape)
print("Shape Y_dev: ", Y_dev.shape)

print("Shape X_test: ", X_test.shape)
print("Shape X_test: ", Y_test.shape, "\n")

print("Training with a total of ", X_train.shape[0], "training examples!")
print("#training examples pyramidal neurons:", np.count_nonzero(Y_train < 1))
print("#training examples interneurons:", Y_train.shape[0] - np.count_nonzero(Y_train < 1))


model = import_fully_connected_ANN(5, [256,256,100,100,2])

model.fit(X_train, Y_train, epochs=5)

print("The model classifies accurately in", model.evaluate(X_dev, Y_dev)[1]*100, "%")

Shape X_train:  (61055, 256)
Shape Y_train:  (61055,)
Shape X_dev:  (10000, 256)
Shape Y_dev:  (10000,)
Shape X_test:  (10000, 256)
Shape X_test:  (10000,) 

Training with a total of  61055 training examples!
#training examples pyramidal neurons: 29181
#training examples interneurons: 31874
Train on 61055 samples
Epoch 1/5
61055/61055 [==============================] - 5s 85us/sample - loss: 0.1448 - accuracy: 0.9790
Epoch 2/5
61055/61055 [==============================] - 5s 76us/sample - loss: 0.0362 - accuracy: 0.9899
Epoch 3/5
61055/61055 [==============================] - 5s 76us/sample - loss: 0.0238 - accuracy: 0.9928
Epoch 4/5
61055/61055 [==============================] - 5s 76us/sample - loss: 0.0216 - accuracy: 0.9939
Epoch 5/5
10000/10000 [==============================] - 0s 45us/sample - loss: 0.0317 - accuracy: 0.9905
The model classifies accurately in 99.04999732971191 %
